In [15]:
import pandas as pd
#透過dict去定義dataframe
a = {'open':[1,2,3],
     'close':[4,5,6]}
df = pd.DataFrame(a)
df.columns
df.index

RangeIndex(start=0, stop=3, step=1)

In [16]:
#透過list 去定義dataframe
b = [[1,2,3],
     [4,5,6],
     [7,8,9],
     [10,11,12]]
df2 = pd.DataFrame(b)
df2_1 =df2.transpose()
df2.columns = ['no1','no2','no3']

In [17]:
# dataframe 常用屬性
# 1.方法（函數）2.屬性（變數）
df2.shape #面積
df2.ndim #二維資料
df2.dtypes #屬性

no1    int64
no2    int64
no3    int64
dtype: object

In [ ]:
#取得欄位資料
type(df2)
df2['no2']
type(df2['no2'])
df2.no2

#取得欄位細節 
df2.info() #物件方法.（函數）
max([1,2,3])#一般函數

df2.describe()
df2.describe([.1,.2,.3])

In [20]:
#資料取得
df2['no2'][1]


5

In [24]:
# locate 定位
df2.loc[2,'no2']# dataframe.loc[index,column]
df2.loc[3,'no3']

12

In [26]:
#iloc integer locate只能用數值來定位
df2.iloc[2,0]   #dataframe.iloc[ index id ,column id]
#新增df欄位 #確保資料程度一致
df2['no4'] = [1,2,3,4]

In [28]:
#篩選資料
df2['no4']      #df2[column]
df2['no4']> 2   #condition
df2_cond1 = df2[ df2['no4']> 2 ] #df2[bulin]

#篩選資料 loc
df2_cond2 = df2.loc[ df2['no4']> 2, 'no3']  #dataframe.loc[ index cond ,column ]

In [ ]:
import yfinance
import matplotlib.pyplot as plt

In [ ]:
data1 = yfinance.download('VOO')
data1.columns
data1.head()
data1.tail()
plt.figure()
data1['Open'].plot()
plt.figure()
data1['Open'].plot(legend=True)
data1['Hign'].plot(legend=True)
data1['Low'].plot(legend=True)
#   pandas dataframe 分布圖
data1['Open'].plot.hist()
#   pandas dataframe 散點圖
data1.plot.scatter('Open','Close')
#   成交量與當日漲跌幅 散點圖
data1['漲跌'] = data1['Close'] - data1['Open']
data1.plot.scatter('Volume','漲跌')

#   群組化
# 1.分群 確定分群依據 
# 2.統計 聚合
#   分群特徵長度 要跟原資料長度相同
data1.index.year 
gp = data1.groupby(data1.index.year) #分群依據
#   統計 每年的最高收盤價
gp['Close'].max()                   #()聚合函數
gp['Volume'].sum()
#   多因子分群
gp2 = data1.groupby([data1.index.year,
                    data1.index.month])
rs = gp2['Volume'].sum()

#   transform
gp2['Volume'].transform('sum')
data1['當月總成交量'] = gp2['Volume'].transform('sum')


def close_stats(group):
    return pd.Series({
        'mean':group['Close'].mean(),
        'Total':group['Close'].sum()
    })

#   groupby apply
gp2.apply(close_stats)

In [ ]:
#   時間序列
data2 = pd.read_csv()

#   去將時間欄位 轉成 python時間格式
pd.to_datetime(data2['Date'])    #轉換Series to datetime
#   將時間欄位 設定為索引
data2 = data2.set_index('Date') 
#   時間序列 更快速依照時間篩選資料
data2.loc['2023'] 
data2.loc['2023-01']
data2.loc['2023-01':'2023-04'] 
 
#   resample
data2.resample('M')['Volume'].sum()
#   將日Ｋ轉月Ｋ
data2.resample('M')['Volume'].agg({'Open':'first'})
#   如何在當天的行數 取得過去的資訊(lag)
data2['昨收'] = data2['Close'].shift(1)
data2['前日收'] = data2['Close'].shift(2)
data2['明日收'] = data2['Close'].shift(-1)

data2['Adj Close'].diff(1)
#   取得同一欄位與前一筆的差 diff
data2['還原報酬率'] = data2['Adj Close'].diff(1)/data2['Close'].shift(1)

#   累計相乘 cumprod
(data2['還原報酬率']+1).cumprod().plot()
#   累積相加 cumsum
#   累積最大 cummax
(data2['還原報酬率']+1).cumprod().cummax.plot()
#   累積最小 cummin
(data2['還原報酬率']+1).cumprod().cummin.plot()
#   移動窗格 rolling window
#   設定完窗格範圍  聚合函數
data2['移動平均線'] = data2.rolling(3)['Close'].mean()

data2['壓力'] = data2.rolling(20)['High'].max()
data2['支撐'] = data2.rolling(20)['Low'].min()
data2['Close'].plot()
data2['壓力'].plot()
data2['支撐'].plot()
#   寫入檔案
data2.to_csv('')



In [ ]:
#pip3 install FinMind
from FinMind.data import DataLoader

api = DataLoader()
# api.login_by_token(api_token='token')
# api.login(user_id='user_id',password='password')
# 取得股權分散表的歷史資料
holding_shares = api.taiwan_stock_holding_shares_per(
    stock_id="2330",
    start_date='2001-01-01',
    end_date='2020-04-12'
)
#   資料結構轉換?
holding_shares['date'] = pd.to_datetime(holding_shares['date'])
holding_shares_1 = holding_shares.pivot(index='date',
                                        columns = 'HoldingSharesLevel',
                                        values = 'percent'
                                        )
#   時間序列合併 前提為 index ==datetime
stock_price = yfinance.download('0050.TW')
#   表格合併 依照欄位或者是索引 concat
#   預設欄位合併
combind_data0 = pd.concat([stock_price,holding_shares_1])
#   設定為index 合併
combind_data = pd.concat([stock_price,holding_shares_1], axis=1)

#   補齊資料
tmp_columns = ['',
    
                ]
#   資料向前補齊na值 ffill
combind_data[tmp_columns] = combind_data[tmp_columns].ffill()

#   資料清洗 
#   isna 是判斷series 裡是為空值
#   notna 是判斷series裡不是為空值
combind_data[combind_data['Open'].isna()]
combind_data[combind_data['Open'].notna()]
combind_data[combind_data['1-999'].notna()]




In [ ]:
#pip install mplfinance
import mplfinance as mpf
#   該函數成功條件
#   1.會自動抓取dataframe 的Open High Low Close Volume
#   2.並且Dataframe一定要是時間序列
mpf.plot(combind_data)
mpf.plot(combind_data,type='candle')
mpf.plot(combind_data,type='candle',style='charles')

mpf.available_styles()

#設定 紅漲綠跌 的樣式
mcolor = mpf.make_marketcolors(up='r', down='g', inherit=True)
mstyle = mpf.make_mpf_style(base_mpf_style='charles', marketcolors=mcolor)
mpf.plot(combind_data, type='candle', style=mstyle)

In [ ]:
# 將股權分散表 去進行計算
# 30張以下 視為小股東
# 30~600張 視為中實戶
# 600張以上 視為大股東
小股東欄位 = ['1-999', '1,000-5,000', '5,001-10,000',
         '10,001-15,000', '15,001-20,000', '20,001-30,000', ]
大股東欄位 = ['600,001-800,000', '800,001-1,000,000', 'more than 1,000,001']
combind_data['小股東'] = combind_data[小股東欄位].sum(axis=1)
combind_data['大股東'] = combind_data[大股東欄位].sum(axis=1)

# 新增附圖的物件 panel由0-9 0是主圖：Ｋ線圖 附圖可畫9個
addp = []       #把附圖裝進箱子
addp.append(mpf.make_addplot(combind_data['小股東'], panel=1))
addp.append(mpf.make_addplot(combind_data['大股東'],panel=2))
mpf.plot(combind_data, type='candle', style=mstyle, addplot=addp)
